To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [2]:
from unsloth import FastModel
import torch
max_seq_length = 2048

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-270m-it",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.6: Fast Gemma3 patching. Transformers: 4.55.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors:   0%|          | 0.00/536M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

We now add LoRA adapters so we only need to update a small amount of parameters!

In [41]:
model = FastModel.get_peft_model(
    model,
    r = 16,  # enough capacity for your dataset
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,  # usually 2x r
    lora_dropout = 0.1,  # small dropout to reduce overfitting
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Making `model.base_model.model.model` require gradients


<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [4]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma3",
)

In [36]:
from datasets import load_dataset

# Load the dataset from the local jsonl file
dataset = load_dataset("json", data_files="unsloth_gemma_dataset.jsonl", split="train")

We now use `convert_to_chatml` to try converting datasets to the correct format for finetuning purposes!

Let's see how row 100 looks like!

In [37]:
dataset[100]

{'conversations': [{'role': 'user',
   'content': "You are a strict running coach. Based on the activity data, respond ONLY with JSON containing two keys: 'summary' and 'reflection'. The summary must be at most 2 sentences, use simple words, and be concise.\n\nActivity Data:\n- Date: 2024-10-09\n- Activity: Run\n- Distance: 1.34 km\n- Pace: 6:53/km\n- Elevation Gain: 10 m\n- Average Heart Rate: N/A\n- Recent Runs:\n- 25.27 km at 5:57/km\n- 12.62 km at 3:08/km\n"},
  {'role': 'assistant',
   'content': '{"summary": "A very short run today at an unacceptably slow pace.", "reflection": "Compared to your recent 3:08/km pace, this 6:53/km is an insult. This pathetic performance shows zero effort and a complete lack of dedication."}'}]}

We now have to apply the chat template for `Gemma3` onto the conversations, and save it to `text`.

In [38]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/197 [00:00<?, ? examples/s]

Let's see how the chat template did!


In [39]:
dataset[100]['text']

'<start_of_turn>user\nYou are a strict running coach. Based on the activity data, respond ONLY with JSON containing two keys: \'summary\' and \'reflection\'. The summary must be at most 2 sentences, use simple words, and be concise.\n\nActivity Data:\n- Date: 2024-10-09\n- Activity: Run\n- Distance: 1.34 km\n- Pace: 6:53/km\n- Elevation Gain: 10 m\n- Average Heart Rate: N/A\n- Recent Runs:\n- 25.27 km at 5:57/km\n- 12.62 km at 3:08/km<end_of_turn>\n<start_of_turn>model\n{"summary": "A very short run today at an unacceptably slow pace.", "reflection": "Compared to your recent 3:08/km pace, this 6:53/km is an insult. This pathetic performance shows zero effort and a complete lack of dedication."}<end_of_turn>\n'

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [47]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 5, # Set this for 1 full training run.
        learning_rate = 2e-5, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir="outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [48]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Let's verify masking the instruction part is done! Let's print the 100th row again.

In [49]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><start_of_turn>user\nYou are a strict running coach. Based on the activity data, respond ONLY with JSON containing two keys: \'summary\' and \'reflection\'. The summary must be at most 2 sentences, use simple words, and be concise.\n\nActivity Data:\n- Date: 2024-10-09\n- Activity: Run\n- Distance: 1.34 km\n- Pace: 6:53/km\n- Elevation Gain: 10 m\n- Average Heart Rate: N/A\n- Recent Runs:\n- 25.27 km at 5:57/km\n- 12.62 km at 3:08/km<end_of_turn>\n<start_of_turn>model\n{"summary": "A very short run today at an unacceptably slow pace.", "reflection": "Compared to your recent 3:08/km pace, this 6:53/km is an insult. This pathetic performance shows zero effort and a complete lack of dedication."}<end_of_turn>\n'

Now let's print the masked out example - you should see only the answer is present:

In [50]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                                                                                                             {"summary": "A very short run today at an unacceptably slow pace.", "reflection": "Compared to your recent 3:08/km pace, this 6:53/km is an insult. This pathetic performance shows zero effort and a complete lack of dedication."}<end_of_turn>\n'

In [51]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
1.279 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [52]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 197 | Num Epochs = 5 | Total steps = 125
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 3,796,992 of 271,895,168 (1.40% trained)


Step,Training Loss
1,3.814100
2,3.533200
3,3.690900
4,3.175300
5,2.990200
6,3.461500
7,3.136100
8,2.965900
9,2.888400
10,3.048800


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

457.9899 seconds used for training.
7.63 minutes used for training.
Peak reserved memory = 4.229 GB.
Peak reserved memory for training = 3.397 GB.
Peak reserved memory % of max memory = 28.689 %.
Peak reserved memory for training % of max memory = 23.045 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [59]:
test_prompt = """You are a strict running coach. Based on the activity data, respond ONLY with JSON containing two keys: 'summary' and 'reflection'. The summary must be at most 2 sentences, use simple words, and be concise.

Activity Data:
- Date: 2024-01-13
- Activity: Run
- Distance: 5.29 km
- Pace: 18:22/km
- Elevation Gain: 17 m
- Average Heart Rate: N/A
- Recent Runs:
- 3.03 km at 4:41/km
- 10.02 km at 5:14/km
"""

In [61]:
messages = [
    {'role': 'user','content':test_prompt},
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
).removeprefix('<bos>')

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 150,
    temperature = 1, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

{"summary": "You ran 5.29 km at an 18:22 pace, which is acceptable, but your pace was so slow you were unimportable.", "reflection": "This pace is a tad slow for your form today. Your last 10km is clearly faster than you did yesterday, and this is clear a serious improvement."}<end_of_turn>


In [34]:
messages = [
    {'role': 'system','content':dataset['conversations'][10][0]['content']},
    {"role" : 'user', 'content' : dataset['conversations'][10][1]['content']}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
).removeprefix('<bos>')

text

'<start_of_turn>user\nYou are a strict running coach. Based on the activity data, respond ONLY with JSON containing two keys: \'summary\' and \'reflection\'. The summary must be at most 2 sentences, use simple words, and be concise.\n\nActivity Data:\n- Date: 2024-01-13\n- Activity: Run\n- Distance: 5.29 km\n- Pace: 20:22/km\n- Elevation Gain: 17 m\n- Average Heart Rate: N/A\n- Recent Runs:\n- 3.03 km at 9:41/km\n- 3.02 km at 6:14/km\n\n\n{"summary": "A 5.29 km run with an extremely slow pace. Your 20:22 per kilometer pace is simply unacceptable.", "reflection": "This 20:22/km pace is pathetic. It\'s a massive regression from your recent runs; show some actual effort."}<end_of_turn>\n<start_of_turn>model\n'

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [19]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/tokenizer.model',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [21]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

==((====))==  Unsloth 2025.8.6: Fast Gemma3 patching. Transformers: 4.55.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/393M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [25]:
# Merge to 16bit
if True:
    model.save_pretrained_merged("lora_model", tokenizer, save_method = "merged_16bit")
# if False: # Pushing to HF Hub
#     model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# # Merge to 4bit
# if True:
#     model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
# if False: # Pushing to HF Hub
#     model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# # Just LoRA adapters
# if True:
#     model.save_pretrained("model")
#     tokenizer.save_pretrained("model")
# if False: # Pushing to HF Hub
#     model.push_to_hub("hf/model", token = "")
#     tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [26]:
if True: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "lora_model",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Unsloth: Updating system package directories
Unsloth: Install GGUF and other packages


RuntimeError: Unsloth: `config.json` does not exist inside `lora_model`.

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>


In [6]:
from datasets import load_dataset

# Load the dataset from the local jsonl file
dataset = load_dataset("json", data_files="unsloth_gemma_dataset.jsonl", split="train")

# Display the first few rows of the dataset
display(dataset)

Dataset({
    features: ['conversations'],
    num_rows: 197
})

In [23]:
!zip -r lora_model.zip lora_model/
from google.colab import files
files.download("lora_model.zip")

  adding: lora_model/ (stored 0%)
  adding: lora_model/added_tokens.json (stored 0%)
  adding: lora_model/tokenizer.model (deflated 52%)
  adding: lora_model/chat_template.jinja (deflated 70%)
  adding: lora_model/adapter_model.safetensors (deflated 8%)
  adding: lora_model/tokenizer_config.json (deflated 97%)
  adding: lora_model/tokenizer.json (deflated 83%)
  adding: lora_model/special_tokens_map.json (deflated 77%)
  adding: lora_model/README.md (deflated 66%)
  adding: lora_model/adapter_config.json (deflated 57%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>